In [ ]:
!pip install yt_dlp pydub transformers langchain langchain-community chromadb ffmpeg-python sentence-transformers


In [1]:
import yt_dlp as youtube_dl
from pydub import AudioSegment
from transformers import pipeline, LlamaForCausalLM, LlamaTokenizer
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import os


/Users/hosung/anaconda3/envs/whisper-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hosung/anaconda3/envs/whisper-rag/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
def download_audio_from_youtube(youtube_url, output_format='wav'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': output_format,
            'preferredquality': '192',
        }],
        'outtmpl': '%(id)s.%(ext)s',
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)
        audio_file = ydl.prepare_filename(info_dict).replace('.webm', f'.{output_format}')
        
    return audio_file

def transcribe_audio_with_pipeline(audio_file):
    transcriber = pipeline(task="automatic-speech-recognition", model="openai/whisper-base")
    result = transcriber(audio_file)
    return result['text']



In [3]:
# 유튜브 URL 입력 받기
youtube_url = "https://youtu.be/W6peueGxp9g"
# 오디오 파일 다운로드 및 변환
audio_file = download_audio_from_youtube(youtube_url)
# 오디오 파일을 텍스트로 변환
transcription = transcribe_audio_with_pipeline(audio_file)

[youtube] Extracting URL: https://youtu.be/W6peueGxp9g
[youtube] W6peueGxp9g: Downloading webpage
[youtube] W6peueGxp9g: Downloading ios player API JSON
[youtube] W6peueGxp9g: Downloading player f9c66401
[youtube] W6peueGxp9g: Downloading m3u8 information
[info] W6peueGxp9g: Downloading 1 format(s): 251
[download] Destination: W6peueGxp9g.webm
[download] 100% of    7.68MiB in 00:00:00 at 10.44MiB/s  
[ExtractAudio] Destination: W6peueGxp9g.wav
Deleting original file W6peueGxp9g.webm (pass -k to keep)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [4]:
print(transcription)

 예전 영상에서 신입부터 5년차까지 뭐가 중요한 수단 얘기를 했는데 그 다음 5년은 본격적으로 커리어의 날개를 피기 위한 준비와 훈련을 하면서 인정도 받아야 되는 중요한 식이라고 생각을 합니다. 그래서 오늘은 커리어 5년에서 10년차까지는 뭐가 중요하고 뭘 챙겨야 되는지 이야기 보려고 합니다. 첫 5년차까지는 내가 뭘 좋아하고 뭘 잘하고 장기적으로는 어떤 길을 걷고 싶은지 찾아야 되는 시기란 얘기를 했어요. 그다음 5년차는 그렇게 찾은 내가 되고 싶은 모습을 본격적으로 만들어가는 시기라고 생각합니다. 집 내 전문성을 쌓는 시기이고 나는 모하는 사람인지를 만들어가는 시기인 거죠. 그럼 첫 번째로 해야 되는 질문은 내가 지금 하고 있는 일이 장기적으로 전문성을 쌓고 싶은 1인이라는 거죠. 첫 5년은 다양한 경험을 하는 것도 좋지만 그 다음 5년은 깊게 들어가야 하기 때문에 필요하다면 부소를 옮기던가 이지글에서 변화를 줘야 되는 거죠. 이런 변화를 주는 과정은 준비가 오래 걸리기 때문에 하루라도 빨리 시작하면 좋은 것 같습니다. 저는 제가 원하는 컬을 위해서 경력 4년 반차 때 대학원 유학을 가서 큰 변화를 주었고 이 과정은 제가 이전 영상에서 자세하게 얘기했으니 참고해주세요. 그렇게 변화를 주었다면 이전의 전문성을 싸악하기 시작해야 돼요. 저는 실리콘 벨리 테크 기업에 취업은 성공을 했는데 막상 이를 시작해보니까 미국 동료를 사이에서 현지인도 아닌 내가 어떤 전문성을 가져가야 될지 고민이 많이 되더라고요. 그러면서 내 전문성을 쌓는 과정은 마치 농구 선수가 전문성을 쌓는 과정과 비슷한 생각을 했어요. 모든 걸 다 자라면 물론 좋지만 3.2 수술 잘 쏘는 선수, 리바운드를 잘하는 선수처럼 내 전문 분야가 있으면 오랫동안 필요한 선수가 되잖아요. 저도 이걸 찾아야겠다는 생각을 했고 제가 찾은 답은 미국 회사에서 아시아 기업들과 일을 잘하는 포지션이었어요. 그래서 아시아 기업과 제유를 하면서 제품을 출시하는 일은 미국 동료들보다는 내가 더 잘할 수 있는 일이 없고 또 

In [ ]:
def correct_text_with_llm(text):
    model_name = "meta-llama/Llama-2-7b-hf"
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    model = LlamaForCausalLM.from_pretrained(model_name)
    corrector = pipeline("text-generation", model=model, tokenizer=tokenizer)
    
    # 텍스트 보정 실행
    corrected_text = corrector(text, max_length=512, num_return_sequences=1)[0]['generated_text']
    return corrected_text

In [ ]:
corrected_transcription = correct_text_with_llm(transcription)

In [ ]:


# RAG 시스템 초기화 함수 정의
def initialize_rag_system(documents):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma.from_texts(texts=documents, embedding=embeddings)
    
    model_name = "meta-llama/Llama-2-7b-hf"  # 사용할 LLaMA 모델 이름
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    model = LlamaForCausalLM.from_pretrained(model_name)
    llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
    llm = HuggingFacePipeline(pipeline=llm_pipeline)
    
    qa_chain = RetrievalQA(llm=llm, retriever=vectorstore.as_retriever())
    return qa_chain

# 텍스트를 문서 리스트로 변환
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_text(corrected_transcription)

# RAG 시스템 초기화
qa_chain = initialize_rag_system(documents)

# 질의 예시
query = "해당 영상의 주요 내용은 무엇인가요?"
response = qa_chain.run(query)

# 결과 출력
print("Query:", query)
print("Response:", response)

# 임시 오디오 파일 삭제
os.remove(audio_file)
